In [1]:
import torch
import torch.nn as nn
from src.models import EndiannessModel
from src.dataset_loaders.ISAdetect_mips_mipsel import create_train_test_dataloaders

# Check for available device
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS")
else:
    device = torch.device("cpu")
    print("Using CPU")

# Create dataloaders
mips_dir = "../../dataset/ISAdetect/ISAdetect_full_dataset/mips"
mipsel_dir = "../../dataset/ISAdetect/ISAdetect_full_dataset/mipsel"
train_loader, test_loader = create_train_test_dataloaders(
    mips_dir=mips_dir, mipsel_dir=mipsel_dir, test_split=0.2
)

# Initialize model and move to device
model = EndiannessModel().to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters())

num_epochs = 5

Using MPS
Epoch 1/5, Step 100, Loss: 32.29051971435547
Epoch 1/5, Loss: 31.258201599121094
Epoch 2/5, Step 100, Loss: 13.791988372802734
Epoch 2/5, Loss: 0.23960690200328827
Epoch 3/5, Step 100, Loss: 0.15108178555965424
Epoch 3/5, Loss: 0.0887766107916832
Epoch 4/5, Step 100, Loss: 0.05474809557199478
Epoch 4/5, Loss: 0.15646892786026
Epoch 5/5, Step 100, Loss: 0.11293213814496994
Epoch 5/5, Loss: 0.0236424021422863
Accuracy: 0.9884


In [ ]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    for i, (batch_x, batch_y) in enumerate(train_loader):
        # Move batch to device
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        optimizer.zero_grad()
        output = model(batch_x)
        targets = batch_y.unsqueeze(1)
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Step {i+1}, Loss: {loss.item()}")
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

In [ ]:
# Test model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for batch_x, batch_y in test_loader:
        # Move batch to device
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        output = model(batch_x)
        predictions = (output >= 0.5).squeeze().float()
        correct += (predictions == batch_y).sum().item()
        total += batch_y.size(0)

    accuracy = correct / total
    print(f"Accuracy: {accuracy:.4f}")